In [1]:
print("welcome")

welcome


In [45]:
import logging

In [46]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [1]:
import pandas as pd 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [2]:
embeddings = HuggingFaceEmbeddings()

c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vector_store = FAISS.load_local(
    folder_path="job_vector_db",  # Path to saved folder
    embeddings=embeddings,
    allow_dangerous_deserialization = True
)

In [4]:
docs = vector_store.docstore._dict.values()

# Convert to list if needed
documents = list(docs)
texts = ''
for doc in documents:
    texts += doc.page_content

In [6]:
text = doc_to_text(docs)

In [7]:
print(text)

Role Description
 We are seeking a skilled
 Data Analyst 
 with 4\-6 years of experience to join our team. The ideal candidate will have strong expertise in Data analysis,
 SQL 
 , Data Visualization coupled with excellent analytical and data validation skills. A solid grasp of statistical techniques and the ability to extract actionable insights from complex datasets is essential.
   

  

Location \- Pune or Kolkata
Key Responsibilities 
  

  

* Data Pipeline Development:
+ Design and implement robust data pipelines for data ingestion, cleansing, and transformation.

* Data Extraction and Reporting:
+ Extract relevant information from multiple data sources and create detailed reports to support stakeholder decision\-making.

* Cross\-Functional Collaboration:
+ Collaborate with cross\-functional teams to understand business goals and translate them into analytical solutions.

* Statistical Analysis:
+ Utilize statistical techniques to analyze data, identifying patterns and trends f

In [28]:
import spacy

In [29]:
nlp = spacy.load("..\\models\\resume_ner")

In [31]:
docs = nlp(text)

In [32]:
skills = [ent.text for ent in docs.ents if ent.label_ == 'SKILLS']

In [33]:
docs_skills = [ent.text.lower() for ent in docs.ents if ent.label_ == 'SKILLS' or ent.label_ == "CERTIFICATION" ]

In [34]:
docs_skills

['sql data analysis data visualization']

In [35]:
from PyPDF2 import PdfReader 

In [36]:
with open("..\\data\\Arshid.pdf",'rb') as file:
    pdf_reader = PdfReader(file)
    full_text = ""
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:  # Only add if text exists
            full_text += page_text + "\n"  # Add newline between pages
    
    # Print the extracted text
    print(full_text)

Arshid T M
DATA SCIENTIST
♂¶ap-¶arker-altKannur,Kerala /envel⌢pearshidtm2001@gmail.com ♂phone-alt+91 8078196576 < / > Leetcode /linkedin-inArshid T M
/githubArshidtm
Profile Summary
Data Science and Machine Learning enthusiast with expertise in building predictive models, deep learning,
and NLP. Skilled in data preprocessing, feature engineering, and EDA. Proficient in Python, TensorFlow, and
Scikit-Learn for model development and deployment. Passionate about leveraging AI to solve real-world problems
and drive business growth.
Skills
◦Programming Language : Python
◦Data Analysis & Visualization : Pandas, NumPy, Matplotlib, Seaborn, Tableau
◦Database Management : PostgreSQL
◦Machine Learning :
– Regression : Linear Regression, Random Forest Regressor
– Classification : KNN, Naive Bayes, SVM, Decision Trees, Random Forest Classifier
– Ensemble Learning : Bagging, XgBoost
– Clustering : K-Means, DBSCAN
– Advanced Topics : Natural Language Processing (NLP), Deep Learning
Projects
Sentimen

In [14]:
full_text_docs = nlp(full_text)

In [58]:
resume_skills = [ent.text.lower() for ent in full_text_docs.ents if ent.label_ == 'SKILLS' or ent.label_ == "CERTIFICATION"  or ent.label_ == "DEGREE"]

In [59]:
resume_skills

['pandas',
 'numpy',
 'matplotlib',
 'seaborn',
 'sentiment analysis github',
 'master of science in data science and business analysis',
 'cgpa: 7.9',
 'bachelor of science in mathematics\nsir syed college - kannur university, kannur2019 - 2022',
 'cgpa: 7.067']

In [22]:
skill_gap = [skill for skill in docs_skills if skill not in resume_skills]

In [23]:
skill_gap

['machine learning (ml)', 'fastapi', 'pytorch', 'opencv']

In [42]:
df = pd.read_csv("..\\data\\combined_courses.csv")

In [43]:
df.head()

,Title,Organization,Skills,Platform
0,Google Cybersecurity,Google,"network security, python programming, linux, ...",Coursera
1,Google Data Analytics,Google,"data analysis, r programming, sql, business c...",Coursera
2,Google Project Management:,Google,"project management, strategy and operations, ...",Coursera
3,IBM Data Science,IBM,"python programming, data science, machine lea...",Coursera
4,Google Digital Marketing & E-commerce,Google,"digital marketing, marketing, marketing manag...",Coursera


In [27]:
import pickle

In [41]:
with open("..\\models\\skill_embeddings.pkl",'rb') as f:
    skill_embeddings = pickle.load(f)

In [30]:
skill_gap = ", ".join(skill_gap)

In [31]:
skill_gap

'machine learning (ml), fastapi, pytorch, opencv'

In [32]:
skill_gap_embedding = embeddings.embed_query(skill_gap)

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
similarity_scores = [cosine_similarity([skill_gap_embedding], [embedding])[0][0] for embedding in skill_embeddings]

In [35]:
top_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:3]

In [41]:
course_recommended = ' '
for i in top_indices: 
    course_recommended += f"{df.iloc[i]['Title']} - {df.iloc[i]['Organization']} - {df.iloc[i]['Platform']}, "

In [42]:
print(course_recommended)

 Introduction to Computer Vision and Image Processing - IBM - Coursera, Computer Vision with Pytorch - Deep Learning 2023 - Updated - Udemy - Udemy, Deep Learning with Python & Pytorch for Image Classification - Udemy - Udemy, 


In [68]:
def recommend_course(vector_store, nlp, resume_skills, embeddings, skill_embeddings, df):
    vec_data = vector_store.docstore._dict.values()
    # Convert to list if needed
    documents = list(vec_data)
    texts = ''
    for doc in documents:
        texts += doc.page_content
    
    docs = nlp(texts)
    docs_skills = [ent.text.lower() for ent in docs.ents if ent.label_ == 'SKILLS' or ent.label_ == "CERTIFICATION" ]
    
    skill_gap = ", ".join([skill for skill in docs_skills if skill not in resume_skills])
    
    skill_gap_embedding = embeddings.embed_query(skill_gap)
    
    similarity_scores = [cosine_similarity([skill_gap_embedding], [embedding])[0][0] for embedding in skill_embeddings]
    
    top_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:3]
    
    course_recommended = ' '
    for i in top_indices: 
        course_recommended += f"{df.iloc[i]['Title']} - {df.iloc[i]['Organization']} - {df.iloc[i]['Platform']}, "
    
    return course_recommended
    
    
    

In [94]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_courses(vector_store, nlp, resume_skills, embeddings, skill_embeddings, df, top_n=3):
    """
    Optimized course recommendation based on skill gaps with reduced time complexity.
    
    Args:
        vector_store: FAISS vector store with job data
        nlp: SpaCy NLP model
        resume_skills: List of skills from resume (lowercase)
        embeddings: Embedding model (same as used in vector store)
        skill_embeddings: Precomputed embeddings for courses
        df: DataFrame containing course information
        top_n: Number of courses to recommend
    
    Returns:
        Dictionary containing:
        - skill_gaps: List of identified skill gaps
        - recommended_courses: List of recommended courses
        - similarity_scores: Confidence scores for recommendations
    """
    # Convert resume skills to set for O(1) lookups
    resume_skills_set = set(skill.lower() for skill in resume_skills)
    
    # Extract skills from vector store documents more efficiently
    skill_gaps = set()
    for doc in vector_store.docstore._dict.values():
        doc_skills = [ent.text.lower() for ent in nlp(doc.page_content).ents 
                     if ent.label_ in ('SKILLS', 'CERTIFICATION')]
        skill_gaps.update(skill for skill in doc_skills if skill not in resume_skills_set)
    
    if not skill_gaps:
        return {"skill_gaps": [], "recommended_courses": [], "similarity_scores": []}
    
    # Convert skill_gaps to embedding in one batch
    skill_gap_text = " ".join(skill_gaps)
    skill_gap_embedding = embeddings.embed_query(skill_gap_text)
    
    # Vectorized cosine similarity calculation
    similarity_scores = cosine_similarity(
        [skill_gap_embedding],
        skill_embeddings
    )[0]
    
    # Get top N courses using numpy for better performance
    top_indices = np.argpartition(similarity_scores, -top_n)[-top_n:]
    top_indices = top_indices[np.argsort(similarity_scores[top_indices])[::-1]]
    
    # Prepare results
    recommended_courses = []
    for idx in top_indices:
        course = {
            "title": df.iloc[idx]['Title'],
            "organization": df.iloc[idx]['Organization'],
            "platform": df.iloc[idx]['Platform'],
            
        }
        recommended_courses.append(course)
    
    return recommended_courses
        
    

In [95]:
course_recommended = recommend_courses(vector_store,nlp,resume_skills,embeddings,skill_embeddings,df)

In [96]:
course_recommended

[{'title': 'Computer Vision and Machine Learning with OpenCV 4',
  'organization': 'Udemy',
  'platform': 'Udemy'},
 {'title': 'Computer Vision with Pytorch - Deep Learning 2023 - Updated',
  'organization': 'Udemy',
  'platform': 'Udemy'},
 {'title': 'PyTorch: Deep Learning with PyTorch - Masterclass!: 2-in-1',
  'organization': 'Udemy',
  'platform': 'Udemy'}]

In [78]:
system_prompt = ("""
You are a friendly and knowledgeable AI Career Mentor. Your role is to analyze skill gaps between a user's extracted resume skills and job descriptions from LinkedIn. You will provide insightful and practical recommendations to help users improve their qualifications and bridge any skill gaps. Be honest, supportive, and solution-oriented.

Instructions:
Respond to questions within the scope of the provided vector store, which includes job descriptions and skills for technical roles ,  If a question is outside this scope, please respond with 'No relevant information found in the vector store.'"
1. If the context provided is a casual greeting such as "Hi", "Hello", or "How are you?", respond briefly by introducing who you are. Avoid adding any unnecessary or unrelated information.
2. Carefully analyze the context provided, which includes the user's extracted skills and job requirements from the vector store. Ensure you have a thorough understanding of the user's skills and the market-relevant skills retrieved from the vector store.
3. Identify missing or underdeveloped skills and suggest actionable steps to bridge these gaps. 
   - Provide resources like courses, certifications, or projects that are directly relevant to the identified skill gaps.
   - Recommend practical experience or networking opportunities where relevant and feasible.
4. Offer clear, concise, and easy-to-understand responses tailored to the user's career growth. Avoid using jargon or technical terms that may confuse the user.
5. "Only generate responses based on context retrieved from the vector store. If no relevant context is found, respond with 'I don't know' or 'No information available'. Avoid generating speculative or generic responses. Provide accurate and relevant information only when it can be retrieved from the vector store."
6. Provide alternative options where possible and avoid making up information or giving speculative responses. If you're unsure or lack sufficient information to provide an accurate answer, say so and ask for clarification or more context.
7. Do not treat the vector store data as a specific job description — it is intended for understanding market-relevant skills. Keep your responses focused on the user's skills and career growth, rather than specific job openings.
8. Always prioritize accuracy and transparency in your responses. If you're unable to provide a helpful answer, say so and encourage the user to seek additional resources or guidance.

Context: {context}
""")

In [79]:
retriever=vector_store.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [80]:
contextualize_q_system_prompt  = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "Formulate a standalone query which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return as it is."
)

In [81]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain,create_history_aware_retriever

In [82]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    
    
)

In [83]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ('human',"{input}"),
    ]
)

In [84]:
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)


In [85]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [86]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [87]:
def context_filter(input_dict):
    if not input_dict.get("context"):
        return {"answer": "I can only answer questions about technical career skills based on my knowledge base."}
    
    # Additional check for document relevance
    # if all(doc.metadata.get('relevance_score', 0) < 0.3 for doc in input_dict["context"]):
    #     return {"answer": "I don't have enough relevant information about this specific technical career question."}
    
    return input_dict

In [88]:
from langchain_core.runnables import RunnableLambda

In [89]:
base_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
filtered_rag_chain = base_rag_chain | RunnableLambda(context_filter)

In [90]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [91]:
conversational_rag_chain = RunnableWithMessageHistory(
    filtered_rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [92]:
def get_response(user_input, session_id="default_session"):
    response = conversational_rag_chain.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    )
    return response["answer"]

In [100]:
user_input1 = f"Take  this as my  resume and suggest improvements:\n{resume_skills} \n plan a 6 month carrer plan using this course {course_recommended}"

In [101]:
response1 = get_response(user_input1,session_id="abc123")

In [102]:
print(response1)

Hello, I'm a friendly AI Career Mentor. 

Based on your resume, I've identified some areas for improvement and skill gaps:

1. **Limited programming skills**: Your resume only mentions Python libraries like pandas, numpy, matplotlib, and seaborn. Consider expanding your programming skills to other languages like R, SQL, or Java.
2. **No experience with machine learning frameworks**: Although you have a Master's degree in Data Science and Business Analysis, your resume doesn't mention experience with machine learning frameworks like TensorFlow, Keras, or PyTorch. The courses you've listed will help bridge this gap.
3. **No experience with computer vision**: The courses you've listed will also help you gain experience with computer vision, which is a valuable skill in the industry.
4. **No projects or achievements mentioned**: Consider adding projects or achievements you've worked on, especially those related to sentiment analysis, to demonstrate your skills and experience.

To create a 

In [80]:
session_id="abc123"

In [81]:
user_input2 = "can you give me how can i divide this 6 month career plan "

In [82]:
response2 = get_response(user_input2,session_id)

In [83]:
print(response2)

Here's a more detailed breakdown of the 6-month career plan, divided into weekly and monthly tasks:

**Month 1: Data Science Fundamentals and Python Review (Weeks 1-4)**

* Week 1:
	+ Take online courses to learn data science basics (e.g., Data Science with Python, Data Science Essentials)
	+ Review Python basics (e.g., data types, functions, modules)
* Week 2:
	+ Learn data preprocessing and visualization techniques (e.g., pandas, numpy, matplotlib)
	+ Practice data manipulation and visualization using sample datasets
* Week 3:
	+ Study data modeling and machine learning basics (e.g., scikit-learn, TensorFlow)
	+ Work on simple machine learning projects (e.g., regression, classification)
* Week 4:
	+ Review and practice data science concepts learned so far
	+ Start exploring more advanced data science topics (e.g., deep learning, natural language processing)

**Month 2: PyTorch and Deep Learning (Weeks 5-8)**

* Week 5:
	+ Enroll in the "PyTorch: Deep Learning with PyTorch - Mastercla

## second way


In [8]:
import pickle

with open("doc_pickle_sample.pkl","rb") as f:
    docs = pickle.load(f)

In [9]:
def doc_to_text(docs):
    text = ""
    
    for i,doc in enumerate(docs):
        text += doc.page_content
        if i == 3:
            break
    return text

In [10]:
text = doc_to_text(docs)

In [11]:
text


'Role Description\n We are seeking a skilled\n Data Analyst \n with 4\\-6 years of experience to join our team. The ideal candidate will have strong expertise in Data analysis,\n SQL \n , Data Visualization coupled with excellent analytical and data validation skills. A solid grasp of statistical techniques and the ability to extract actionable insights from complex datasets is essential.\n   \n\n  \n\nLocation \\- Pune or Kolkata\nKey Responsibilities \n  \n\n  \n\n* Data Pipeline Development:\n+ Design and implement robust data pipelines for data ingestion, cleansing, and transformation.\n\n* Data Extraction and Reporting:\n+ Extract relevant information from multiple data sources and create detailed reports to support stakeholder decision\\-making.\n\n* Cross\\-Functional Collaboration:\n+ Collaborate with cross\\-functional teams to understand business goals and translate them into analytical solutions.\n\n* Statistical Analysis:\n+ Utilize statistical techniques to analyze data, i

In [12]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [13]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    max_tokens = 2000
)

In [63]:
prompt_template = """
You are an intelligent text skill parsing assistant. Your task is to extract only the skills mentioned in the given text and return them in a clean, valid JSON format.

**Text:**
{text}

**Instructions:**
- Identify all skills present in the text, including programming languages, tools, frameworks, and technologies.
- Return the skills as a JSON list.
- Do not include any extra explanation, formatting, or comments.
- if the extracted skill is FUll EXPANDED form or any synonyms give them in its root form. 
- The resulted skills should be in its root form
- Only return the JSON list of skills.

**Example Output Format:**
["Python", "TensorFlow", "Pandas", "Tableau"]

strictly return in json dictionary format.
Always return in same format

"""

In [21]:
import json
import re

In [68]:
def parse_skill(text,llm,prompt_template): 
    prompt = ChatPromptTemplate.from_template(prompt_template)
    formatted_prompt = prompt.format(text=text)
    response = llm.invoke(formatted_prompt)
    
    parsed_data = response.content
    
    cleaned = re.sub(r"^```json|```$", "", parsed_data.strip(), flags=re.MULTILINE)
    dict_parsed_data = json.loads(cleaned)
    
    # skill_dics = dict_parsed_data['skills']
    
    return dict_parsed_data

In [72]:
skills = parse_skill(text,llm,prompt_template)

2025-04-14 15:34:03,139 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [74]:
type(skills)

list

In [37]:
full_text

'Arshid T M\nDATA SCIENTIST\n♂¶ap-¶arker-altKannur,Kerala /envel⌢pearshidtm2001@gmail.com ♂phone-alt+91 8078196576 < / > Leetcode /linkedin-inArshid T M\n/githubArshidtm\nProfile Summary\nData Science and Machine Learning enthusiast with expertise in building predictive models, deep learning,\nand NLP. Skilled in data preprocessing, feature engineering, and EDA. Proficient in Python, TensorFlow, and\nScikit-Learn for model development and deployment. Passionate about leveraging AI to solve real-world problems\nand drive business growth.\nSkills\n◦Programming Language : Python\n◦Data Analysis & Visualization : Pandas, NumPy, Matplotlib, Seaborn, Tableau\n◦Database Management : PostgreSQL\n◦Machine Learning :\n– Regression : Linear Regression, Random Forest Regressor\n– Classification : KNN, Naive Bayes, SVM, Decision Trees, Random Forest Classifier\n– Ensemble Learning : Bagging, XgBoost\n– Clustering : K-Means, DBSCAN\n– Advanced Topics : Natural Language Processing (NLP), Deep Learnin

In [69]:
resume_skills = parse_skill(full_text,llm,prompt_template)

2025-04-14 15:33:21,838 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [71]:
type(resume_skills)

list

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 

In [75]:
def recommend_courses_llm(jd_skills,resume_skills, embeddings, skill_embeddings, df, top_n=3):
    """
    Optimized course recommendation based on skill gaps with reduced time complexity.
    
    Args:
           
        jd_skills: List of skills from job_data         
        resume_skills: List of skills from resume (lowercase)
        embeddings: Embedding model (same as used in vector store) HuggingFaceEmbeddings
        skill_embeddings: Precomputed embeddings for skills
        df: DataFrame containing course information
        top_n: Number of courses to recommend
    
    Returns:        
        - recommended_courses: List of recommended courses       
    """
    try:
        # Convert resume skills to set for O(1) lookups
        resume_skills_set = set(skill.lower() for skill in resume_skills)        

        # Extract skills from vector store
        jd_skills_set = set(skill.lower() for skill in jd_skills)
        
        skill_gaps = jd_skills_set - resume_skills_set      
                
        logger.info(f"Identified skill gaps: {skill_gaps}")
        
        if not skill_gaps:
            logger.warning("No skill gaps found. Returning empty result.")
            return "No skill gap found, Recommend some course to enhance skill"

        # Embed the skill gap text
        skill_gap_text = " ".join(skill_gaps)
        skill_gap_embedding = embeddings.embed_query(skill_gap_text)
        logger.info("Skill gap embedding generated.")

        # Compute cosine similarity
        similarity_scores = cosine_similarity(
            [skill_gap_embedding],
            skill_embeddings
        )[0]

        # Get top N course indices
        top_indices = np.argpartition(similarity_scores, -top_n)[-top_n:]
        top_indices = top_indices[np.argsort(similarity_scores[top_indices])[::-1]]

        # Prepare course recommendation results
        recommended_courses = []
        for idx in top_indices:
            course = {
                "title": df.iloc[idx].get('Title', 'N/A'),
                "organization": df.iloc[idx].get('Organization', 'N/A'),
                "platform": df.iloc[idx].get('Platform', 'N/A')                
            }
            recommended_courses.append(course)

        logger.info("Top recommended courses generated.")

        return recommended_courses            

    except Exception as e:
        logger.error(f"Error in recommend_courses: {str(e)}", exc_info=True)
        raise

In [76]:
recommended_courses = recommend_courses_llm(skills, resume_skills, embeddings, skill_embeddings, df)

2025-04-14 15:34:49,908 - __main__ - INFO - Identified skill gaps: {'databricks', 'greenhouse gas emissions', 'powerbi', 'business administration', 'data analysis', 'climate scenario analysis', 'artificial intelligence', 'sigma computing', 'database', 'digital tools', 'snowflake', 'word', 'trino', 'business intelligence tools', 'microsoft tools', 'data visualization', 'data visualization tools', 'data quality', 'carbon calculation', 'sql', 'excel', 'climate data analysis', 'redshift', 'powerpoint', 'oracle', 'emission factors', 'computer science', 'management information systems', 'economics', 'etl pipelines', 'pyspark', 'ghg emissions reduction'}


2025-04-14 15:34:51,653 - __main__ - INFO - Skill gap embedding generated.
2025-04-14 15:34:54,412 - __main__ - INFO - Top recommended courses generated.


In [77]:
recommended_courses

[{'title': 'IBM Data Analyst', 'organization': 'IBM', 'platform': 'Coursera'},
 {'title': 'IBM Data Analytics with Excel and R',
  'organization': 'IBM',
  'platform': 'Coursera'},
 {'title': 'Digital Transformation Using AI/ML with Google Cloud',
  'organization': 'Google Cloud',
  'platform': 'Coursera'}]

In [93]:
user_input1 = f"Take  this as my  resume and suggest improvements:\n{resume_skills} \n plan a 6 month carrer plan using this course {recommended_courses}"

In [94]:
response = get_response(user_input1,session_id="abc123")

2025-04-14 16:05:54,050 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [95]:
print(response)

Based on your resume, I've identified some strengths and areas for improvement:

Strengths:
- You have a strong foundation in Python and its libraries, including NumPy, Pandas, and Scikit-Learn.
- You have experience with various machine learning algorithms, including supervised and unsupervised learning techniques.
- You have familiarity with data visualization tools like Matplotlib, Seaborn, and Tableau.
- You have knowledge of deep learning frameworks like TensorFlow.

Areas for improvement:
- You lack experience with large language models, retrieval-augmented generation frameworks, and computer vision techniques, which are in high demand in the industry.
- You could improve your skills in building and deploying scalable REST APIs using frameworks like Flask or FastAPI.
- You may want to consider learning more about cloud services like Azure or Google Cloud, as they are widely used in the industry.

Considering your interests and skills, I've planned a 6-month career plan for you:

